# XXZ model on Qiskit and IonQ simulator

In [4]:
from qiskit import QuantumCircuit, transpile
from qiskit_ionq import IonQProvider

In [5]:
# Load your API key from an environment variable named MY_IONQ_API_KEY
my_api_key = "pOiUVlzriOoF2wX1kp3lIqid1OMhwXZ5"
provider = IonQProvider(my_api_key)

In [8]:
from qiskit.quantum_info import Pauli, SparsePauliOp
import numpy as np

def xxz_hamiltonian(n, delta):
    """Returns the XXZ model Hamiltonian for n qubits and a given delta in Qiskit."""
    
    # Initialize lists to store the Pauli strings and coefficients
    pauli_strings = []
    coefficients = []

    for i in range(n):
        # XX term (X_i * X_{i+1})
        x_term = ['I'] * n
        x_term[i] = 'X'
        x_term[(i + 1)%n] = 'X'
        pauli_strings.append(''.join(x_term))
        coefficients.append(1.0)
        
        # YY term (Y_i * Y_{i+1})
        y_term = ['I'] * n
        y_term[i] = 'Y'
        y_term[(i + 1)%n] = 'Y'
        pauli_strings.append(''.join(y_term))
        coefficients.append(1.0)
        
        # ZZ term (Z_i * Z_{i+1})
        z_term = ['I'] * n
        z_term[i] = 'Z'
        z_term[(i + 1)%n] = 'Z'
        pauli_strings.append(''.join(z_term))
        coefficients.append(delta)

    # Create the SparsePauliOp object
    paulis = [Pauli(p) for p in pauli_strings]
    hamiltonian = SparsePauliOp(paulis, np.array(coefficients))
    
    return hamiltonian

In [16]:
# Step 2: Prepare the quantum circuit (example: simple circuit for a 2-qubit system)
n_qubits = 3
circuit = QuantumCircuit(n_qubits)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()

In [17]:
n_qubits = 3
delta_value = 0.5
hamiltonian = xxz_hamiltonian(n_qubits, delta_value)
print(hamiltonian)

SparsePauliOp(['XXI', 'YYI', 'ZZI', 'IXX', 'IYY', 'IZZ', 'XIX', 'YIY', 'ZIZ'],
              coeffs=[1. +0.j, 1. +0.j, 0.5+0.j, 1. +0.j, 1. +0.j, 0.5+0.j, 1. +0.j, 1. +0.j,
 0.5+0.j])


In [20]:
# Set simulator backend
simulator_backend = provider.get_backend("ionq_simulator")
# Run the circuit on IonQ's platform:
job = simulator_backend.run(circuit, shots=10000)

# Print the counts
print(job.get_counts())

{'000': 4981, '011': 5019}
